In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from pyspark.sql.types import ArrayType, FloatType, IntegerType, StringType, StructType, StructField
from pyspark.ml.feature import SQLTransformer
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, concat, udf, col, array
from tensorflow.keras.models import load_model
from pyspark.ml.linalg import Vectors, VectorUDT
import numpy as np
from PIL import Image
from pyspark.ml import PipelineModel

2024-12-25 23:29:23.886990: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-25 23:29:23.897966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735162163.911083   33088 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735162163.914454   33088 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 23:29:23.927606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
spark: SparkSession = SparkSession.builder.appName("Traffic Signs Classification Streaming") \
    .master("local[*]") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.python.worker.memory", "12g") \
    .config("spark.executor.pyspark.memory", "12g") \
    .config("spark.rpc.message.maxSize", "128") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.sql.streaming.checkpointLocation", "./tmp") \
    .getOrCreate()

dataset_path = "./dataset"

schema = StructType([
    StructField("Width", IntegerType(), True),
    StructField("Height", IntegerType(), True),
    StructField("Roi.X1", IntegerType(), True),
    StructField("Roi.Y1", IntegerType(), True),
    StructField("Roi.X2", IntegerType(), True),
    StructField("Roi.Y2", IntegerType(), True),
    StructField("ClassId", IntegerType(), True),
    StructField("Path", StringType(), True)
])

input_stream = spark.readStream.option("header", "true").schema(schema).csv(os.path.join(dataset_path, "streaming"))
input_stream = input_stream.withColumn("Path", concat(lit(dataset_path + "/"), input_stream["Path"])).dropna(subset=["ClassId"])


your 131072x1 screen size is bogus. expect trouble
24/12/25 23:29:24 WARN Utils: Your hostname, DESKTOP-SMHNFU4 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/25 23:29:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 23:29:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def process_image(img_path, roi_x1, roi_y1, roi_x2, roi_y2):
    try:
        img = Image.open(img_path)
        cropped_img = img.crop((roi_x1, roi_y1, roi_x2, roi_y2))
        resized_img = np.array(cropped_img.resize((32, 32), resample=Image.Resampling.LANCZOS))
        return (resized_img.flatten() / 255.0).tolist()
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        return [0.0] * (32 * 32 * 3)

def image_to_vector(img_features):
    return Vectors.dense(img_features)

spark.udf.register("process_image", process_image, ArrayType(FloatType()))
spark.udf.register("image_to_vector", image_to_vector, VectorUDT())

<function __main__.image_to_vector(img_features)>

In [4]:
lrModel = PipelineModel.load("best-model-lr")
rfModel = PipelineModel.load("best-model-rf")

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888



In [5]:
lrPredictionsStream = lrModel.transform(input_stream)

outputStream = lrPredictionsStream.select("Path", "ClassId", "prediction").writeStream.option("path", "output/lr").outputMode("append").trigger(once=True).option("header", "true").format("csv").start()
%time outputStream.awaitTermination()

24/12/25 23:29:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/25 23:29:40 WARN DAGScheduler: Broadcasting large task binary with size 1289.0 KiB
Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888



CPU times: user 14.7 ms, sys: 444 μs, total: 15.1 ms
Wall time: 53.8 s


In [6]:
rfPredictionsStream = rfModel.transform(input_stream)

outputStream = rfPredictionsStream.select("Path", "ClassId", "prediction").writeStream.option("path", "output/rf").outputMode("append").trigger(once=True).option("header", "true").format("csv").start()
%time outputStream.awaitTermination()

24/12/25 23:30:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/25 23:30:37 WARN DAGScheduler: Broadcasting large task binary with size 318.3 MiB
Current mem limits: 12884901888 of max 12884901888                  (0 + 5) / 5]

Current mem limits: 12884901888 of max 12884901888

Current mem limits: 12884901888 of max 12884901888

Current mem limits: 12884901888 of max 12884901888

Current mem limits: 12884901888 of max 12884901888



CPU times: user 4.67 ms, sys: 9.77 ms, total: 14.4 ms
Wall time: 48.2 s


In [7]:
# too slow

# tflite_model_path = "best_model.tflite"
# broadcast_model_path = spark.sparkContext.broadcast(tflite_model_path)

# def predict_image_class(img_path, roi_x1, roi_y1, roi_x2, roi_y2):
#   try:
#     img = Image.open(img_path)
#     cropped_img = img.crop((roi_x1, roi_y1, roi_x2, roi_y2))
#     resized_img = np.array(cropped_img.resize((32, 32), resample=Image.Resampling.LANCZOS)) / 255.0
#     resized_img = np.expand_dims(resized_img, axis=0).astype(np.float32)
    
#     model_path = broadcast_model_path.value
#     interpreter = tf.lite.Interpreter(model_path=model_path)
#     interpreter.allocate_tensors()
#     input_details = interpreter.get_input_details()
#     output_details = interpreter.get_output_details()
    
    
#     interpreter.set_tensor(input_details[0]['index'], resized_img)
#     interpreter.invoke()
#     output_data = interpreter.get_tensor(output_details[0]['index'])
    
#     predicted_class = np.argmax(output_data)
#     return int(predicted_class)

    
#   except Exception as e:
#     print(f"Error processing image {img_path}: {e}")
#     raise e
  
  
# predict_image_class_udf = udf(predict_image_class, IntegerType())

# kerasPredictionsStream = input_stream.withColumn("prediction", predict_image_class_udf("Path", "`Roi.X1`", "`Roi.Y1`", "`Roi.X2`", "`Roi.Y2`"))

# outputStream = kerasPredictionsStream.select("Path", "ClassId", "prediction").writeStream.option("path", "output/keras").outputMode("append").trigger(once=True).format("csv").start()
# outputStream.awaitTermination()

In [8]:
from pandas import DataFrame


def process_batch(batch_df, batch_id):
    model_path = "best_model.tflite"
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    batch_pd: DataFrame = batch_df.toPandas()

    def predict_image_class(row):
        try:
            img_path = row["Path"]
            roi_x1, roi_y1, roi_x2, roi_y2 = row["Roi.X1"], row["Roi.Y1"], row["Roi.X2"], row["Roi.Y2"]
            
            img = Image.open(img_path)
            cropped_img = img.crop((roi_x1, roi_y1, roi_x2, roi_y2))
            resized_img = np.array(cropped_img.resize((32, 32), resample=Image.Resampling.LANCZOS)) / 255.0
            resized_img = np.expand_dims(resized_img, axis=0).astype(np.float32)

            interpreter.set_tensor(input_details[0]['index'], resized_img)
            interpreter.invoke()
            
            output_data = interpreter.get_tensor(output_details[0]['index'])
            predicted_class = int(np.argmax(output_data))

            return predicted_class
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            return None

    batch_pd["prediction"] = batch_pd.apply(predict_image_class, axis=1)
    batch_pd = batch_pd.filter(items=["Path", "ClassId", "prediction"])
    result_df = spark.createDataFrame(batch_pd)
    result_df.write.format("csv").option("header", "true").option("path", "output/keras").mode("append").save()

query = input_stream.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("append") \
    .trigger(once=True) \
    .start()

%time query.awaitTermination()


24/12/25 23:31:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current mem limits: -1 of max -1

Setting mem limits to 12884901888 of max 12884901888

Current 

CPU times: user 51.9 s, sys: 7.6 s, total: 59.5 s
Wall time: 3min 50s


In [17]:
lr_results = spark.read.csv("output/lr", header=True)
rf_results = spark.read.csv("output/rf", header=True)
keras_results = spark.read.csv("output/keras", header=True)

lr_total = lr_results.count()
rf_total = rf_results.count()
keras_total = keras_results.count()

print("\n\nTotal count")
print(f"lr {lr_total}\nrf {rf_total}\nkeras {keras_total}")

lr_accuracy = lr_results.filter(lr_results["ClassId"] == lr_results["prediction"].cast(IntegerType())).count() / lr_total
rf_accuracy = rf_results.filter(rf_results["ClassId"] == rf_results["prediction"].cast(IntegerType())).count() / rf_total
keras_accuracy = keras_results.filter(keras_results["ClassId"] == keras_results["prediction"]).count() / keras_total

print("\n\nAccuracy")
print(f"lr {lr_accuracy}\nrf {rf_accuracy}\nkeras {keras_accuracy}")




Total count
lr 63150
rf 63150
keras 63150


Accuracy
lr 0.8592240696753761
rf 0.8188440221694379
keras 0.97458432304038
